In [0]:
try:
  # %tensorflow_version only exists in Colab.
  #%tensorflow_version 2.x
  %tensorflow_version 1.x  
except Exception:
  pass

# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
import datetime
from os import makedirs
from os.path import exists, join
import numpy as np

import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

from keras.callbacks import TensorBoard

In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [0]:
log_dir= "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

if not exists(log_dir):
    makedirs(log_dir)

# set parameters:
max_features = 5000 # Put it up to 10k?
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250 # play with this too
kernel_size = 3
hidden_dims = 250
epochs = 2

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

# save class labels to disk to color data points in TensorBoard accordingly
with open(join(log_dir, 'metadata.tsv'), 'w') as f:
    np.savetxt(f, y_test)

tensorboard = TensorBoard(log_dir=log_dir,
                          histogram_freq=1, 
                          embeddings_freq=1,
                          embeddings_layer_names=['features'],
                          embeddings_metadata='metadata.tsv',
                          embeddings_data=x_test)

Loading data...
17465344/17464789 [==============================] - 2s 0us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)


In [0]:
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
#model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims, name='features'))
#model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          #callbacks=[tensorboard], # Enabling callbacks crashes training. TODO
          epochs=epochs,
          validation_data=(x_test, y_test))






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 25000 samples, validate on 25000 samples
Epoch 1/2





25000/25000 [==============================] - 14s 549us/step - loss: 0.3709 - acc: 0.8272 - val_loss: 0.2620 - val_acc: 0.8924
Epoch 2/2
25000/25000 [==============================] - 5s 217us/step - loss: 0.1904 - acc: 0.9250 - val_loss: 0.2638 - val_acc: 0.8932


In [0]:
model2 = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model2.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
#model2.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model2.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='selu',
                 strides=1))
# we use max pooling:
model2.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model2.add(Dense(hidden_dims, name='features'))
#model2.add(Dropout(0.2))
model2.add(Activation('selu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model2.add(Dense(1))
model2.add(Activation('sigmoid'))

model2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model2.fit(x_train, y_train,
          batch_size=batch_size,
          #callbacks=[tensorboard],
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 7s 293us/step - loss: 0.3671 - acc: 0.8246 - val_loss: 0.2700 - val_acc: 0.8877
Epoch 2/2
25000/25000 [==============================] - 7s 280us/step - loss: 0.1882 - acc: 0.9268 - val_loss: 0.2642 - val_acc: 0.8926
